## Solução baseada em 2 artigos:

- https://medium.com/trustyou-engineering/topic-modelling-with-pyspark-and-spark-nlp-a99d063f1a6e
- https://medium.com/analytics-vidhya/distributed-topic-modelling-using-spark-nlp-and-spark-mllib-lda-6db3f06a4da3

# Como foi feito

O objetivo é descobrir os principais tópicos das revisões presentes nos dados. Uma ideia inicial era apenas observar as palavras mais frequentes, porém, em algumas pesquisas, encontramos um algoritmo de modelagem de tópicos chamado de _LDA (Latent Dirichlet Allocation)_, que de forma resumida, agrupa palavras com maior probabilidade de aparecerem juntas em tópicos, em outras palavras, essa técnica é capaz de "deduzir" os tópicos presentes em um corpo de texto qualquer, utilizando probabilidade.

Por essa razão, utilizamos esta técnica, pois informamos o número desejado de tópicos e o algoritmo irá detectar esses n tópicos. O número de tópicos é vital, pois ao optar por um número grande de tópicos, você pode ter tópicos com muitas palavras em comum, contudo, menos tópicos também pode implicar no seu tópico ser geral demais e portanto não fornecer nenhuma informação útil para análise.



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, concat_ws, split, hour, explode
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA


spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.4") \
    .getOrCreate()

df = spark.read.json('file:///home/ec2-user/eiffel-tower-reviews.json').select('text')
df.show()

:: loading settings :: url = jar:file:/home/ec2-user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache

22/12/08 20:23:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
+--------------------+
|                text|
+--------------------+
|This is the most ...|
|My significant ot...|
|We had a tour to ...|
|Visited with my w...|
|We went in the ni...|
|Dont hesitate and...|
|I enjoyed the tow...|
|Read through the ...|
|This by far was o...|
|Something you hav...|
|The views are bea...|
|Worth spending a ...|
|Took the tour to ...|
|A fantastic fusio...|
|Whatever you do i...|
|Not to miss..beau...|
|We visited in the...|
|Go for sunset and...|
|We booked weeks a...|
|Eiffel Tower is j...|
+--------------------+
only showing top 20 rows



# Preprocessamentos

Antes de aplicar o LDA, aplicamos um pipeline de processamentos nos dados visando normalizar os dados, remover stopwords (palavra muito comuns de um idioma como artigos, preposições, etc). Essa etapa é importante para garantir que o modelo tenha maior precisão.

In [2]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")
# Split sentence to tokens(array)
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
# clean unwanted characters and garbage
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")
# remove stopwords
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)
# stem the words to bring them to the root form.
stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")
# Finisher is the most important annotator. Spark NLP adds its own structure when we convert each row in the dataframe to document. Finisher helps us to bring back the expected structure viz. array of tokens.
finisher = Finisher() \
    .setInputCols(["stem"]) \
    .setOutputCols(["tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)
# We build a ml pipeline so that each phase can be executed in sequence. This pipeline can also be used to test the model. 
nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher])
# train the pipeline
nlp_model = nlp_pipeline.fit(df)
# apply the pipeline to transform dataframe.
processed_df  = nlp_model.transform(df)
# nlp pipeline create intermediary columns that we dont need. So lets select the columns that we need
tokens_df = processed_df.select('text', 'tokens')
tokens_df.show()

+--------------------+--------------------+
|                text|              tokens|
+--------------------+--------------------+
|This is the most ...|[busiest, atttact...|
|My significant ot...|[signific, drunke...|
|We had a tour to ...|[tour, eiffel, to...|
|Visited with my w...|  [visit, wife, son]|
|We went in the ni...|[went, night, pm,...|
|Dont hesitate and...|[dont, hesit, got...|
|I enjoyed the tow...|[enjoi, tower, ki...|
|Read through the ...|[read, histori, e...|
|This by far was o...|[far, favourit, p...|
|Something you hav...|[someth, visit, p...|
|The views are bea...|[view, beauti, wo...|
|Worth spending a ...|[worth, spend, mi...|
|Took the tour to ...|[took, tour, top,...|
|A fantastic fusio...|[fantast, fusion,...|
|Whatever you do i...|[whatev, afford, ...|
|Not to miss..beau...|[missbeauti, plac...|
|We visited in the...|[visit, afternoon...|
|Go for sunset and...|[go, sunset, stai...|
|We booked weeks a...|[book, week, ahea...|
|Eiffel Tower is j...|[eiffel, t

In [3]:
cv = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=500, minDF=3.0)
# train the model
cv_model = cv.fit(tokens_df)
# transform the data. Output column name will be features.
vectorized_tokens = cv_model.transform(tokens_df)

Com os dados processados e vetorizados, podemos aplicar o algoritmo LDA para agrupar os dados. Utilizamos aqui n = 4 para obter 4 grupos.

In [4]:
num_topics = 4
lda = LDA(k=num_topics, maxIter=15)
model = lda.fit(vectorized_tokens)

22/12/08 20:24:12 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/12/08 20:24:12 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [5]:
vocab = cv_model.vocabulary
topics = model.describeTopics()   
topics_rdd = topics.rdd
topics_words = topics_rdd \
       .map(lambda row: row['termIndices']) \
       .map(lambda idx_list: [vocab[idx] for idx in idx_list]) \
       .collect()
for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)
    


topic: 0
*************************
on
tower
place
visit
pari
eiffel
see
must
landmark
great
*************************
topic: 1
*************************
tower
eiffel
pari
see
go
visit
time
night
view
must
*************************
topic: 2
*************************
ticket
get
line
top
view
time
go
wait
tower
queue
*************************
topic: 3
*************************
tower
view
top
go
visit
eiffel
pari
night
dai
amaz
*************************


Acima, podemos ver os 4 tópicos que foram derivados a partir dos dados. Note que há alguma semântica nos tópicos, por exemplo, o primeiro tópico aparentemente trata sobre a importância de visitar a torre eiffel por ser um grande ponto em Paris, enquanto o segundo tópico fala sobre uma bela vista noturna e o terceiro tópico a respeito de filas, aparentemente para entrar na torre.